In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd 
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm


from sklearn.metrics import mean_squared_error, r2_score
import os

In [4]:
train = pd.read_csv('train_prepared.csv', index_col=[0], parse_dates=[0])
#test = pd.read_csv('test_prepared.csv', index_col=[0], parse_dates=[0])

train.head()

FileNotFoundError: [Errno 2] File b'train_prepared.csv' does not exist: b'train_prepared.csv'

In [ ]:
ts = train['Passengers']
plt.figure(figsize=(12, 10))
plt.plot(ts)

In [ ]:
fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(ts, lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(ts, lags=40, ax=ax2)

In [ ]:
plt.figure(figsize=(12, 10))
ts_log = np.log1p(ts)
plt.plot(ts_log)

In [ ]:
plt.figure(figsize=(12, 10))
ma = ts_log.rolling(12).mean()
plt.plot(ts_log)
plt.plot(ma)

In [ ]:
plt.figure(figsize=(12, 10))
ts_log_ma_diff = ts_log-ma
ts_log_ma_diff.dropna(inplace=True)
plt.plot(ts_log_ma_diff)

In [ ]:
plt.figure(figsize=(12, 10))
exp_avg = ts_log.ewm(12).mean()
plt.plot(ts_log)
plt.plot(exp_avg)

In [ ]:
plt.figure(figsize=(12, 10))
ts_log_ewma_diff = ts_log-exp_avg
ts_log_ewma_diff.dropna(inplace=True)
plt.plot(ts_log_ewma_diff)

In [ ]:
plt.figure(figsize=(12, 10))
ts_log_diff = ts_log - ts_log.shift()
ts_log_diff.dropna(inplace=True)
plt.plot(ts_log_diff)

In [ ]:
from statsmodels.tsa.stattools import adfuller

def test_ts_stationarity(ts):
    ma = ts.rolling(12).mean()
    std = ts.rolling(12).std()
    plt.figure(figsize=(12, 10))
    orig = plt.plot(ts, color='blue', label='Original')
    mean = plt.plot(ma, color='red', label='Rolling mean')
    std = plt.plot(std, color='black', label='Rolling std')
    plt.legend(loc='best')
    plt.title("Rolling mean & std")
    plt.show()
    #adf test
    print("Results of DF test:")
    dftest = adfuller(ts, autolag='AIC')
    df_test = pd.DataFrame(dftest[:-1])
    df_test.index = ['Test statistic', 'p_value', 'lags', 'num observations', 'critical values']
    print(df_test)

In [ ]:
test_ts_stationarity(ts)

In [ ]:
test_ts_stationarity(ts_log_ewma_diff)

In [ ]:
test_ts_stationarity(ts_log_diff)

In [ ]:
test_ts_stationarity(ts_log_ma_diff)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

dec = seasonal_decompose(ts_log)
trend = dec.trend
seasonal = dec.seasonal
residuals = dec.resid

plt.figure(figsize=(12,10))
plt.subplot(411)
plt.plot(ts_log, label='Original')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.subplot(413)
plt.plot(seasonal, label='Seasonal')
plt.subplot(414)
plt.plot(residuals, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

In [ ]:
test_ts_stationarity(residuals.dropna())

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

model = ARIMA(ts_log, order=(2,1,0))
results_AR = model.fit(disp=-1)
plt.figure(figsize=(10,8))
plt.plot(ts_log_diff)
plt.plot(results_AR.fittedvalues, color='black')
plt.title('RSS: %.4f'%sum((results_AR.fittedvalues-ts_log_diff)**2))

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(10)

CV_scores_AR = []
for train_index, test_index in tscv.split(ts_log):
    model = ARIMA(ts_log[train_index], order=(2,1,0))
    results_AR = model.fit(disp=-1)
    preds = results_AR.forecast(steps=len(test_index))
    print(ts_log[test_index])
    CV_scores_AR.append(r2_score(ts_log[test_index], preds[0]))

In [ ]:
CV_scores_AR

Сохраним обученную модель авторегрессии

In [ ]:
import dill

In [ ]:
model = ARIMA(ts_log, order=(2,1,0))
results_AR = model.fit(disp=-1)

with open("./models/AR.model", "wb") as f:
    dill.dump(results_AR, f)

Moving Average

In [ ]:
model = ARIMA(ts_log, order=(0,1,2))
results_AR = model.fit(disp=-1)
plt.figure(figsize=(10,8))
plt.plot(ts_log_diff)
plt.plot(results_AR.fittedvalues, color='black')
plt.title('RSS: %.4f'%sum((results_AR.fittedvalues-ts_log_diff)**2))

tscv = TimeSeriesSplit(10)

CV_scores_MA = []
for train_index, test_index in tscv.split(ts_log):
    model = ARIMA(ts_log[train_index], order=(0,1,2))
    results_AR = model.fit(disp=-1)
    preds = results_AR.forecast(steps=len(test_index))
    print(ts_log[test_index])
    CV_scores_MA.append(r2_score(ts_log[test_index], preds[0]))

In [ ]:
CV_scores_MA

In [ ]:
model = ARIMA(ts_log, order=(0,1,2))
results_AR = model.fit(disp=-1)

with open("./models/MA.model", "wb") as f:
    dill.dump(results_AR, f)

### ARIMA

In [ ]:
model = ARIMA(ts_log, order=(2,1,2))
results_AR = model.fit(disp=-1)
plt.figure(figsize=(10,8))
plt.plot(ts_log_diff)
plt.plot(results_AR.fittedvalues, color='black')
plt.title('RSS: %.4f'%sum((results_AR.fittedvalues-ts_log_diff)**2))

tscv = TimeSeriesSplit(10)

CV_scores_ARIMA = []
for train_index, test_index in tscv.split(ts_log):
    model = ARIMA(ts_log[train_index], order=(2,1,2))
    results_AR = model.fit(disp=-1)
    preds = results_AR.forecast(steps=len(test_index))
    print(ts_log[test_index])
    CV_scores_ARIMA.append(r2_score(ts_log[test_index], preds[0]))

In [ ]:
CV_scores_ARIMA

In [ ]:
model = ARIMA(ts_log, order=(2,1,2))
results_AR = model.fit(disp=-1)

with open("./models/ARIMA.model", "wb") as f:
    dill.dump(results_AR, f)

Сохраним метрики

In [ ]:
pd.DataFrame({'AR': CV_scores_AR, 
              'MA': CV_scores_MA,
              'ARIMA': CV_scores_ARIMA})

In [ ]:
pd.DataFrame({'AR': CV_scores_AR, 
              'MA': CV_scores_MA,
              'ARIMA': CV_scores_ARIMA}).to_csv("./models/CV_ARIMA.csv")